In [8]:
from utils.utils import gait_dataloader
from utils.utils import init_model,  patients_idx_to_cycles_idx, find_cycles_idx_by_patient_idx
from sklearn.model_selection import train_test_split

from utils.utils import init_model,accuracy_end_to_end, GridSearchCV

import numpy as np

from sklearn.svm import SVC

import pandas as pd

from sklearn.metrics import accuracy_score

def load_dataset_v1(dir_dataset,channel_first=True,flatten=True,shuffle=False,random_state=0):
    index = [0,1,2,3,4,5,6,7,8,9,14,15,16,17,18,19,20,21,22,23,24,29]
    
    npzfile = np.load(dir_dataset + 'Neuro_0.npz')
    
    x_train_all = npzfile['Input']
    y_train = npzfile['Output']
    y_train = np.ravel(y_train)
    x_train = x_train_all[:,:,index]
    if channel_first:
        x_train = x_train.transpose(0,2,1)
    
    npzfile = np.load(dir_dataset + 'Ortho_1.npz')
    x_val_all = npzfile['Input']
    y_val = npzfile['Output']
    y_val = np.ravel(y_val)
    x_val = x_val_all[:,:,index]
    if channel_first:
        x_val = x_val.transpose(0,2,1)
    
    npzfile = np.load(dir_dataset + 'NeuroMusuc_2.npz')
    x_test_all = npzfile['Input']
    y_test = npzfile['Output']
    y_test = np.ravel(y_test)
    x_test = x_test_all[:,:,index]
    if channel_first:
        x_test = x_test.transpose(0,2,1)
    
    if flatten == True:
        x_train = x_train.reshape([x_train.shape[0],-1])
        x_val = x_val.reshape([x_val.shape[0],-1])
        x_test = x_test.reshape([x_test.shape[0],-1])
    
    nb_classes = len(np.unique(np.concatenate((y_train, y_val,y_test), axis=0)))
    
    return x_train, y_train ,x_val, y_val,x_test, y_test, nb_classes

dataset = "archives/AQM/Neuro-Ortho-Neuromuscu/"

x_0, y_0 ,x_1, y_1, x_2, y_2, nb_classes = load_dataset_v1(dataset,channel_first=True,flatten=True,shuffle=False,random_state=0)
# x_after, y_after ,x_before, y_before, nb_classes = load_dataset_v1(dataset,channel_first=True,flatten=False,shuffle=False,random_state=0)

X_d = np.concatenate([x_0,x_1,x_2])
# # X_d = X_d.reshape([len(X_d),-1])
y_d = np.concatenate([y_0,y_1,y_2])
# import  pandas as pd

data_0 = pd.read_csv('DataNeuro_0.csv', header=None)
idx_0 = data_0[1].to_numpy()

# print(after_idx)

data_1 = pd.read_csv('DataOrtho_1.csv', header=None)
idx_1 = data_1[1].to_numpy()
idx_1 = idx_1 + idx_0[-1]

data_2 = pd.read_csv('DataNeuroMuscu_2.csv', header=None)
idx_2 = data_2[1].to_numpy()
idx_2 = idx_2 + idx_1[-1]

cycle_end_idx = np.concatenate([idx_0,idx_1,idx_2])
print(cycle_end_idx)
print(X_d.shape)

[ 22  43  59  73  91 114 140 152 168 182 202 219 238 260 278 300 320 351
 366 378 399 413 424 444 460 480 504 516 528 539 551 568 578 594 633 650
 708 741 760 781 801 825 839 854 864 877 886 903 927 939]
(939, 2222)


In [9]:
len(cycle_end_idx)

50

In [10]:
nb_classes

3

In [11]:
X_d.shape

(939, 2222)

In [4]:
patient_index_range = np.arange(len(idx_0)+len(idx_1)+len(idx_2))
patient_class = np.concatenate([np.zeros(len(idx_0)), np.ones(len(idx_1)),np.ones(len(idx_2))+1])

x_train, x_test, y_train, y_test = train_test_split(patient_index_range, patient_class, test_size=0.4, stratify=patient_class,random_state=0)


x_val, x_test, y_val, y_test = train_test_split(x_test,y_test, test_size=0.75, random_state=2, stratify=y_test)
print(np.unique(y_train,return_counts=True))
print(np.unique(y_val,return_counts=True))
print(np.unique(y_test,return_counts=True))

(array([0., 1., 2.]), array([12, 10,  8]))
(array([0., 1., 2.]), array([2, 1, 2]))
(array([0., 1., 2.]), array([6, 5, 4]))


In [7]:
print(len(patients_idx_to_cycles_idx(x_train[np. where(y_train == 0)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_train[np. where(y_train == 1)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_train[np. where(y_train == 2)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_train,cycle_end_idx)))
print()

print(len(patients_idx_to_cycles_idx(x_val[np. where(y_val == 0)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_val[np. where(y_val == 1)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_val[np. where(y_val == 2)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_val,cycle_end_idx)))
print()

print(len(patients_idx_to_cycles_idx(x_test[np. where(y_test == 0)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_test[np. where(y_test == 1)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_test[np. where(y_test == 2)],cycle_end_idx)))
print(len(patients_idx_to_cycles_idx(x_test,cycle_end_idx)))
print()

238
151
131
520

34
20
36
90

106
101
122
329



In [47]:
238+34+106

378

In [35]:
151+20+101

272

In [36]:
131+36+122

289

In [8]:
378+272+289

939

In [38]:
# before -> 0
y_d.shape

(939,)

In [10]:
from sklearn.model_selection import StratifiedKFold

X = np.concatenate([x_train,x_val])
y = np.concatenate([y_train,y_val])
kf = StratifiedKFold(n_splits=7)

for train_index, test_index in kf.split(X,y):
    print(len(test_index))
    print(np.unique(y[test_index], return_counts=True))
    print(len(patients_idx_to_cycles_idx(X[test_index][np. where(y[test_index] == 0)],cycle_end_idx)))
    print(len(patients_idx_to_cycles_idx(X[test_index][np. where(y[test_index] == 1)],cycle_end_idx)))
    print(len(patients_idx_to_cycles_idx(X[test_index][np. where(y[test_index] == 2)],cycle_end_idx)))

5
(array([0., 1., 2.]), array([2, 1, 2]))
38
17
26
5
(array([0., 1., 2.]), array([2, 1, 2]))
39
20
33
5
(array([0., 1., 2.]), array([2, 1, 2]))
57
11
24
5
(array([0., 1., 2.]), array([2, 2, 1]))
32
37
24
5
(array([0., 1., 2.]), array([2, 2, 1]))
34
29
24
5
(array([0., 1., 2.]), array([2, 2, 1]))
38
27
19
5
(array([0., 1., 2.]), array([2, 2, 1]))
34
30
17


In [50]:
tuned_parameters = [
    {"kernel": ["rbf"], "gamma": [1e-3, 1e-4,1e-5,1e-6], "C": [1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
    {'C': [1, 10, 100, 1000], 'gamma': [1e-3, 1e-4,1e-5,1e-6], 'kernel': ['poly']},
    {'C': [1, 10, 100, 1000], 'gamma': [1e-3, 1e-4,1e-5,1e-6], 'kernel': ['sigmoid']},
]

In [51]:
model_GSCV = GridSearchCV(model=SVC,para_list=tuned_parameters,n_splits=7)

In [8]:
isinstance(np.array([1,2,3]),np.ndarray)

True

In [52]:
# new train tdvspc
model_GSCV.fit(np.concatenate([x_train,x_val]).reshape(-1,1),np.concatenate([y_train,y_val]),X_d,y_d,cycle_end_idx)

True
Multi-process: 10
parameter: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
accuracy: 0.4456626744779904 ± 0.08045388709967582

parameter: {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
accuracy: 0.4456626744779904 ± 0.08045388709967582

parameter: {'C': 1, 'gamma': 1e-05, 'kernel': 'rbf'}
accuracy: 0.5013490103508458 ± 0.1464636426733301

parameter: {'C': 1, 'gamma': 1e-06, 'kernel': 'rbf'}
accuracy: 0.5125039185963793 ± 0.11435721226696824

parameter: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
accuracy: 0.4456626744779904 ± 0.08045388709967582

parameter: {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
accuracy: 0.4456626744779904 ± 0.08045388709967582

parameter: {'C': 10, 'gamma': 1e-05, 'kernel': 'rbf'}
accuracy: 0.5025412524761929 ± 0.15299710910627443

parameter: {'C': 10, 'gamma': 1e-06, 'kernel': 'rbf'}
accuracy: 0.5406996139849759 ± 0.11668882040351909

parameter: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
accuracy: 0.4456626744779904 ± 0.08045388709967582

parameter: {'C': 100, 'ga

In [12]:
accuracy_end_to_end(SVC(**{'C': 100, 'gamma': 1e-6, 'kernel': 'rbf'}),X_d[patients_idx_to_cycles_idx(x_train,cycle_end_idx)], y_d[patients_idx_to_cycles_idx(x_train,cycle_end_idx)],X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)],y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)])

0.8145896656534954

In [13]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)],SVC(**{'C': 100, 'gamma': 1e-6, 'kernel': 'rbf'}).fit(X_d[patients_idx_to_cycles_idx(x_train,cycle_end_idx)], y_d[patients_idx_to_cycles_idx(x_train,cycle_end_idx)]).predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]))

array([[ 93,  11,   2],
       [ 29,  60,  12],
       [  7,   0, 115]])

In [14]:
pre_list = SVC(**{'C': 100, 'gamma': 1e-6, 'kernel': 'rbf'}).fit(X_d[patients_idx_to_cycles_idx(x_train,cycle_end_idx)], y_d[patients_idx_to_cycles_idx(x_train,cycle_end_idx)]).predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)])

In [15]:
from utils.utils import *
error_patient_cycle(x_test,y_test,y_d,cycle_end_idx,pre_list)

-----------------------------
Subject No. 0
cycle_n= 58
n_error= 0

r 0
l 0
pre_list [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
y_true [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
y_test[i] 2.0
majority: 2
well predict for no error
-----------------------------
Subject No. 1
cycle_n= 21
n_error= 7

r 0
l 0
pre_list [0 2 0 2 2 2 0 2 2 0 2 2 2 2 2 2 2 0 2 0 0]
y_true [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
y_test[i] 2.0
majority: 2
well predict for class 2
-----------------------------
Subject No. 2
cycle_n= 22
n_error= 1

r 0
l 1
pre_list [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
y_true [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y_test[i] 0.0
majority: 0
well predict for class 0
-----------------------------
Subject No. 3
cycle_n= 24
n_error= 0

r 0
l 0
pre_list [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
y_true [1 1 1 1 1

In [50]:
# cum_list = np.cumsum([len(find_cycles_idx_by_patient_idx(i,cycle_end_idx)) for i in x_test])

# n_subject_0 = 0
# n_subject_1 = 0
# n_subject_2 = 0
# n_cycle = 0
# n_patient_well_pre = 0


# for i in range(len(x_test)):
#     print('Subject No.',i)
#     print('cycle_n=',len(find_cycles_idx_by_patient_idx(i,(cum_list))))
#     compre_list,compre_list_count = np.unique(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][find_cycles_idx_by_patient_idx(i,cum_list)].astype(int) == pre_list[find_cycles_idx_by_patient_idx(i,cum_list)].astype(int),return_counts=True)
    
#     n_error = 0
#     if False in compre_list:
#         n_error = compre_list_count[np.where(compre_list==False)[0][0]]
#     print('n_error=',n_error)
# #     print(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][find_cycles_idx_by_patient_idx(i,cum_list)].astype(int) == pre_list[find_cycles_idx_by_patient_idx(i,cum_list)].astype(int))
#     print(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][find_cycles_idx_by_patient_idx(i,cum_list)].astype(int))
#     print("pre_list",pre_list[find_cycles_idx_by_patient_idx(i,(cum_list))])
#     print()
#     if n_error:
#         if y_test[i] == 1:
#              n_subject_1+=1

#         elif y_test[i] == 0:
#             n_subject_0+=1
            
#         elif y_test[i] == 2:
#             n_subject_2+=1

#         n_cycle += n_error
        
#     print('n_error',n_error)
#     print('n_error==0',n_error == 0)

#     if y_test[i] == 1 and np.argmax(np.bincount(pre_list[find_cycles_idx_by_patient_idx(i,cum_list)].astype(int)))==1 and n_error!=0:
#         n_patient_well_pre+=1

# #             print(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)][find_cycles_idx_by_patient_idx(i,cum_list)])
# #             print(np.unique(pre_list[find_cycles_idx_by_patient_idx(i,cum_list)].numpy()))
#         print('print 1')

#     elif y_test[i] == 0 and np.argmax(np.bincount(pre_list[find_cycles_idx_by_patient_idx(i,cum_list)].astype(int)))==0 and n_error!=0:
#         n_patient_well_pre+=1

#         print('print 0')

#     elif y_test[i].astype(int) == 2 and np.argmax(np.bincount(pre_list[find_cycles_idx_by_patient_idx(i,cum_list)].astype(int)))==2 and n_error!=0:
#         n_patient_well_pre+=1
#         print('print 2')

#     elif n_error==0:
#         n_patient_well_pre+=1

#         print('print O err')


# print('- Origin -')    
# print('n_subject_2',len(y_test[y_test==2]))
# print('n_subject_1:',len(y_test[y_test==1]))
# print('n_subject_0:',len(y_test[y_test==0]))
# print('n_subject:',len(y_test))
# print('n_patient_well_pre_majority:',n_patient_well_pre)
# print('n_cycles:',len(patients_idx_to_cycles_idx(x_test,cycle_end_idx)))

# print('- Error -')    
# print('n_subject_2:',n_subject_2)
# print('n_subject_1:',n_subject_1)
# print('n_subject_0:',n_subject_0)

# print('n_subject:',n_subject_0+n_subject_1+n_subject_2)
# print('n_cycles:',n_cycle)

Subject No. 0
cycle_n= 58
n_error= 0
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
pre_list [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]

n_error 0
n_error==0 True
print O err
Subject No. 1
cycle_n= 21
n_error= 7
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
pre_list [0. 2. 0. 2. 2. 2. 0. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 0. 2. 0. 0.]

n_error 7
n_error==0 False
print 2
Subject No. 2
cycle_n= 22
n_error= 1
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
pre_list [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

n_error 1
n_error==0 False
print 0
Subject No. 3
cycle_n= 24
n_error= 0
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
pre_list [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

n_error 0
n_error==0 True
print O err
Subject No. 4
cycle_n=